__David Sivieri__  
__CSC8200 Health Informatics & EHealth Systems__  
__Final Project__  
__Feb 24, 2020__

## Data Preprocessing

In this notebook, I will go through the steps of data cleaning and pre-processing.  This includes handling missing data, removing unneeded features, and scaling data.

As I go through these steps in this notebook, I will try and provide some rationale for my decisions.

In [1]:
import re
import math
import sys
import numpy as np
import pandas as pd

from itertools import chain
import matplotlib.pyplot as plt
import seaborn as sns
import csv
from pprint import pprint
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import explained_variance_score, mean_squared_error, r2_score, mean_absolute_error
from scipy import stats
from sklearn.model_selection import RepeatedKFold
import scipy
import sklearn

import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)

pd.set_option("display.max_rows", 1000000)
pd.set_option("display.max_columns", 1000000)
pd.set_option("display.width", 1000000)
pd.set_option("display.max_colwidth", 1000000)
pd.set_option("display.max_info_columns", 1000)
np.set_printoptions(threshold=sys.maxsize)

from IPython.core.display import display, HTML

display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
%%HTML
<style type="text/css">
    table.dataframe td, table.dataframe th {
        border-style: solid;
        border: 1px solid lightgray;
    }
    tr:nth-child(even) {
        background: #eee;
    }
    th {
       background-color: lemonchiffon;
       color: black;
}
</style>

In [3]:
df = pd.read_pickle("diabetic_data.pkl")

In [4]:
df.shape

(101766, 50)

**Here I first replace the databases symbol for missing data: "?" with pandas NAN.  Then I analyze what percentage of each column is NAN.  If it is greater than 20% NAN, then I drop the column.  In most of the dropped cases, the % of missing data was quite high - too high to be useful in the model**

In [5]:
df.replace("?", np.nan, inplace=True)

In [6]:
drop_cols = df.columns[(df.isnull().sum() / len(df)) > 0.2]
print(drop_cols)

Index(['weight', 'payer_code', 'medical_specialty'], dtype='object')


In [7]:
df.drop(drop_cols, axis=1, inplace=True)
df.drop(["age"], axis=1, inplace=True)

df["race"].fillna("unknown", inplace=True)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 101766 entries, (135, 24437208) to (189502619, 371459060)
Data columns (total 46 columns):
race                        101766 non-null object
gender                      101766 non-null object
admission_type_id           101766 non-null int64
discharge_disposition_id    101766 non-null int64
admission_source_id         101766 non-null int64
time_in_hospital            101766 non-null int64
num_lab_procedures          101766 non-null int64
num_procedures              101766 non-null int64
num_medications             101766 non-null int64
number_outpatient           101766 non-null int64
number_emergency            101766 non-null int64
number_inpatient            101766 non-null int64
diag_1                      101745 non-null object
diag_2                      101408 non-null object
diag_3                      100343 non-null object
number_diagnoses            101766 non-null int64
max_glu_serum               101766 non-null object
A1C

**Here I drop rows relating to a patient dying.  Leaving these in would make no sense, as these patients cannt be readmitted**

In [9]:
discharge_codes_to_drop = [11, 19, 20, 21]
drop_idxs = df[df.discharge_disposition_id.isin(discharge_codes_to_drop)].index
df.drop(drop_idxs, inplace=True)

**Here I drop the small % of rows where the gender is unknown**

In [10]:
drop_idxs = df[df.gender == "Unknown/Invalid"].index
df.drop(drop_idxs, inplace=True)

In [11]:
df.shape

(100111, 46)

**Here I convert some numeric data into strings so it will be treated as categorical and not numeric by the models.  These are integers, but their order has no meaning.  If these were left as integers, the model may assume the magnitude of difference between values had some meaning.  Theses are just IDs**

In [12]:
id_cols = ["admission_type_id", "discharge_disposition_id", "admission_source_id"]
df[id_cols] = df[id_cols].astype("str")

df["readmitted_less_30"] = df["readmitted_less_30"].astype("str")

In [13]:
df.nunique()

race                          6
gender                        2
admission_type_id             8
discharge_disposition_id     23
admission_source_id          17
time_in_hospital             14
num_lab_procedures          118
num_procedures                7
num_medications              75
number_outpatient            39
number_emergency             33
number_inpatient             21
diag_1                      715
diag_2                      747
diag_3                      787
number_diagnoses             16
max_glu_serum                 4
A1Cresult                     4
metformin                     4
repaglinide                   4
nateglinide                   4
chlorpropamide                4
glimepiride                   4
acetohexamide                 2
glipizide                     4
glyburide                     4
tolbutamide                   2
pioglitazone                  4
rosiglitazone                 4
acarbose                      4
miglitol                      4
troglita

**Here, I isolate all of the medication columns.  These are indiviual columns for each medicine for each encounter.  To simplify the model, I decided to replace these columns with the total number of medications taken by each patient and the total number of changes of medication during the visit.**

In [14]:
medicine_cols = df.loc[:, "metformin":"metformin-pioglitazone"].columns
diag_cols = df.loc[:, "diag_1":"diag_3"].columns

In [15]:
df["num_med_changes"] = df[medicine_cols].replace({"No": 0, "Steady": 0, "Up": 1, "Down": 1}).sum(axis=1)
df["num_meds_total"] = df[medicine_cols].replace({"No": 0, "Steady": 1, "Up": 1, "Down": 1}).sum(axis=1)
df.drop(medicine_cols, axis=1, inplace=True)

df.drop(diag_cols, axis=1, inplace=True)

In [16]:
df.nunique()

race                          6
gender                        2
admission_type_id             8
discharge_disposition_id     23
admission_source_id          17
time_in_hospital             14
num_lab_procedures          118
num_procedures                7
num_medications              75
number_outpatient            39
number_emergency             33
number_inpatient             21
number_diagnoses             16
max_glu_serum                 4
A1Cresult                     4
change                        2
diabetesMed                   2
readmitted                    3
readmitted_less_30            2
imputed_age                  10
num_med_changes               5
num_meds_total                7
dtype: int64

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 100111 entries, (135, 24437208) to (189502619, 371459060)
Data columns (total 22 columns):
race                        100111 non-null object
gender                      100111 non-null object
admission_type_id           100111 non-null object
discharge_disposition_id    100111 non-null object
admission_source_id         100111 non-null object
time_in_hospital            100111 non-null int64
num_lab_procedures          100111 non-null int64
num_procedures              100111 non-null int64
num_medications             100111 non-null int64
number_outpatient           100111 non-null int64
number_emergency            100111 non-null int64
number_inpatient            100111 non-null int64
number_diagnoses            100111 non-null int64
max_glu_serum               100111 non-null object
A1Cresult                   100111 non-null object
change                      100111 non-null object
diabetesMed                 100111 non-null object


In [18]:
def generate_val_cnts(input_df, col_header):
    df_temp = input_df.value_counts().to_frame(name="Count").join(input_df.value_counts(normalize=True).to_frame(name="Proportion"))
    df_temp.Proportion = (df_temp.Proportion * 100).map("{:.2f}%".format)
    df_temp.index.names = [col_header]
    return df_temp

**The following two charts list frequencies of the remaining admission IDs and discharge IDs**

In [19]:
generate_val_cnts(df.admission_type_id, "admission_type_id")

,Count,Proportion
admission_type_id,,
1,52882,52.82%
3,18738,18.72%
2,18226,18.21%
6,5227,5.22%
5,4690,4.68%
8,320,0.32%
7,18,0.02%
4,10,0.01%


In [20]:
generate_val_cnts(df.discharge_disposition_id, "discharge_disposition_id")

,Count,Proportion
discharge_disposition_id,,
1,60232,60.17%
3,13954,13.94%
6,12902,12.89%
18,3691,3.69%
2,2128,2.13%
22,1992,1.99%
5,1184,1.18%
25,989,0.99%
4,815,0.81%


**To further simplify the model, I collapse the columns "number_outpatient", "number_emergency", "number_inpatient" into one, that is the sum of all 3**

In [21]:
hosp_cols = ["number_outpatient", "number_emergency", "number_inpatient"]

df["all_hospitalizations"] = df[hosp_cols].sum(axis=1)

df.drop(hosp_cols, axis=1, inplace=True)

In [22]:
df.head(20)

race  gender admission_type_id discharge_disposition_id admission_source_id  time_in_hospital  num_lab_procedures  num_procedures  num_medications  number_diagnoses max_glu_serum A1Cresult change diabetesMed readmitted readmitted_less_30  imputed_age  num_med_changes  num_meds_total  all_hospitalizations
patient_nbr encounter_id                                                                                                                                                                                                                                                                                                                              
135         24437208            Caucasian  Female                 2                        1                   1                 8                  77               6               33                 8          None      None     Ch         Yes        <30                  1           55                1               3                     0
            26264286            Caucasian  Female                 1                        1                   7                 3                  31               1               14                 5          None      None     Ch         Yes        >30                  0           55                0               2                     1
378         29758806            Caucasian  Female                 3                        1                   1                 2                  49               1               11                 3          None      None     No          No         NO                  0           55                0               0                     0
729         189899286           Caucasian  Female                 1                        3                   7                 4                  68               2               23                 9          None        >7     No         Yes         NO                  0           85                0               1                     0
774         64331490            Caucasian  Female                 1                        1                   7                 3                  46               0               20                 9          None        >8     Ch         Yes         NO                  0           85                0               2                     0
927         14824206      AfricanAmerican  Female                 1                        1                   7                 5                  49               0                5                 3          None      None     No         Yes         NO                  0           35                0               1                     0
1152        8380170       AfricanAmerican  Female                 1                        1                   7                 6                  43               2               13                 2          None      None     No         Yes        >30                  0           55                0               1                     1
            30180318      AfricanAmerican  Female                 1                        1                   7                 6                  45               4               15                 6          None      None     Ch         Yes        >30                  0           55                1               1                     2
            55533660      AfricanAmerican  Female                 1                        1                   7                10                  54               2               19                 9          None      None     No         Yes        >30                  0           65                0               1                     1
            80742510      AfricanAmerican  Female                 1                        1                   7                 8                  30               1               16                 2          None      None     No         Yes        >30         

**Here, I create a new column named "num_enc_to_date".  This column keeps a cumulative count of the number of encounters that each patient has had with the hospital.  So, if one person had 5 encounters, then the row corresponding to the 5th encounter would have the value of "5" here.  I was attempting to keep some cumulative tally, so the model could weigh how frequent a user of hospital services this patient is.**

In [23]:
df["num_enc_to_date"] = df.groupby(["patient_nbr"]).cumcount() + 1
df.num_enc_to_date.head(20)

patient_nbr  encounter_id
135          24437208        1
             26264286        2
378          29758806        1
729          189899286       1
774          64331490        1
927          14824206        1
1152         8380170         1
             30180318        2
             55533660        3
             80742510        4
             83281464        5
1305         66197028        1
1314         60254142        1
             70190028        2
             70601076        3
1629         171414378       1
2025         14897280        1
3069         36469686        1
4302         85907514        1
5220         7981038         1
Name: num_enc_to_date, dtype: int64

**Here, I turn each of the columns that have categorical data to an explicit numpy "category" data type.  This will ensure that during one-hot encoding, they are treated like categorical data and not numeric data.**

In [24]:
categorical_cols = df.select_dtypes(include="object").columns
df[categorical_cols] = df[categorical_cols].astype("category")

for col_name in df._get_numeric_data().columns:
    df[col_name] = pd.to_numeric(df[col_name], downcast="integer")

df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 100111 entries, (135, 24437208) to (189502619, 371459060)
Data columns (total 21 columns):
race                        100111 non-null category
gender                      100111 non-null category
admission_type_id           100111 non-null category
discharge_disposition_id    100111 non-null category
admission_source_id         100111 non-null category
time_in_hospital            100111 non-null int8
num_lab_procedures          100111 non-null int16
num_procedures              100111 non-null int8
num_medications             100111 non-null int8
number_diagnoses            100111 non-null int8
max_glu_serum               100111 non-null category
A1Cresult                   100111 non-null category
change                      100111 non-null category
diabetesMed                 100111 non-null category
readmitted                  100111 non-null category
readmitted_less_30          100111 non-null category
imputed_age                 10

**Now start analysis of the numeric columns.  In this next few blocks, I will look for highly skewed distributions and perform a log transform if necessary. This technique was used Usman et al[2]**

In [25]:
numeric_cols = list(df._get_numeric_data().columns)
numeric_cols

['time_in_hospital',
 'num_lab_procedures',
 'num_procedures',
 'num_medications',
 'number_diagnoses',
 'imputed_age',
 'num_med_changes',
 'num_meds_total',
 'all_hospitalizations',
 'num_enc_to_date']

In [26]:
mean = df[numeric_cols].mean().rename("mean").to_frame()
std = df[numeric_cols].std().rename("std")
skew = df[numeric_cols].skew().rename("skew")
kurtosis = df[numeric_cols].kurtosis().rename("kurtosis")

mean.join(std).join(skew).join(kurtosis)

,mean,std,skew,kurtosis
time_in_hospital,4.389677,2.974515,1.137913,0.871125
num_lab_procedures,42.943573,19.620796,-0.241447,-0.253211
num_procedures,1.330683,1.700270,1.326176,0.891327
num_medications,15.981810,8.092532,1.333163,3.523961
number_diagnoses,7.409206,1.938265,-0.867642,-0.109317
imputed_age,65.830778,15.947611,-0.626668,0.274183
num_med_changes,0.287551,0.487864,1.425305,1.433645
num_meds_total,1.185564,0.921620,0.675078,0.277231
all_hospitalizations,1.200627,2.292800,5.334664,67.777120
num_enc_to_date,1.627833,1.527544,5.953253,64.983783


**"all_hospitalizations", "num_enc_to_date" both had a skew much higher than 2, so I decided to log transform those columns and look at again**

In [27]:
cols_log_transform = ["all_hospitalizations", "num_enc_to_date"]
df[cols_log_transform] = np.log1p(df[cols_log_transform])

In [28]:
mean = df[numeric_cols].mean().rename("mean").to_frame()
std = df[numeric_cols].std().rename("std")
skew = df[numeric_cols].skew().rename("skew")
kurtosis = df[numeric_cols].kurtosis().rename("kurtosis")

mean.join(std).join(skew).join(kurtosis)

,mean,std,skew,kurtosis
time_in_hospital,4.389678,2.974294,1.137913,0.871125
num_lab_procedures,42.943573,19.620953,-0.241447,-0.253212
num_procedures,1.330683,1.700024,1.326176,0.891327
num_medications,15.981811,8.092546,1.333163,3.523960
number_diagnoses,7.409206,1.938279,-0.867642,-0.109318
imputed_age,65.830780,15.947184,-0.626668,0.274183
num_med_changes,0.287551,0.487832,1.425305,1.433645
num_meds_total,1.185564,0.921673,0.675078,0.277231
all_hospitalizations,0.513761,0.662588,1.118968,0.546821
num_enc_to_date,0.885449,0.350781,2.205675,5.669979


**Here, I apply scaling to all the numeric columns.  This ensures the mean of every numeric column will have a mean value 0 and standard deviation of 1.**

In [29]:
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df[numeric_cols])
df[numeric_cols] = scaled_data

df[numeric_cols].mean()

time_in_hospital        1.578119e-15
num_lab_procedures      5.533044e-16
num_procedures         -8.945108e-16
num_medications         2.571837e-15
number_diagnoses        9.649961e-15
imputed_age             3.009635e-14
num_med_changes        -6.119797e-14
num_meds_total          2.326626e-14
all_hospitalizations    3.862823e-14
num_enc_to_date        -2.265688e-14
dtype: float64

**Here I look at every numeric value in every row.  I any row has an outlier(>=3 std dev from mean) value for its respective column, then that row is removed.  This ends up removing 8% of all rows**

In [30]:
print("Length before removing outliers:", len(df))
df = df[(np.abs(stats.zscore(df[numeric_cols])) < 3).all(axis=1)]
print("Length after removing outliers:", len(df))

Length before removing outliers: 100111
Length after removing outliers: 91679


**Since I already re-coded readmitted as a new column, I'll drop the original 3 valued column**

In [31]:
df.drop(["readmitted"], axis=1, inplace=True)

**Finally, I'll perform one-hot encoding and review results**

In [32]:
categorical_cols = list(df.select_dtypes(include="category").columns)
categorical_cols.remove("readmitted_less_30")

df = pd.get_dummies(df, columns=categorical_cols)
df.readmitted_less_30 = df.readmitted_less_30.astype("int")

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 91679 entries, (135, 24437208) to (189502619, 371459060)
Data columns (total 79 columns):
time_in_hospital               91679 non-null float64
num_lab_procedures             91679 non-null float64
num_procedures                 91679 non-null float64
num_medications                91679 non-null float64
number_diagnoses               91679 non-null float64
readmitted_less_30             91679 non-null int32
imputed_age                    91679 non-null float64
num_med_changes                91679 non-null float64
num_meds_total                 91679 non-null float64
all_hospitalizations           91679 non-null float64
num_enc_to_date                91679 non-null float64
race_AfricanAmerican           91679 non-null uint8
race_Asian                     91679 non-null uint8
race_Caucasian                 91679 non-null uint8
race_Hispanic                  91679 non-null uint8
race_Other                     91679 non-null uint8
race_unk

In [34]:
df.to_pickle("diabetic_data_processed.pkl")